In [ ]:
import pandas as pd
import numpy as np
import geopandas  as gpd
import altair as alt
from datetime import timedelta
from datetime import datetime as dt
import datetime
import json
from altair import datum

alt.data_transformers.disable_max_rows()

In [ ]:

#highlighting important NPIs in WA
data = {'date': [ "2020-03-23", "2020-06-01", "2020-11-18", "2021-02-14"], 'event':[ "Stay at home", "Stay at home lifted", "Closing restaurants", "Reopening restaurants"]}

npidf = pd.DataFrame(data)
npidf.date = pd.to_datetime(npidf.date)

rule = alt.Chart(npidf).mark_rule(
    color="black",
    strokeWidth=2, 
    opacity = 0.3
).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=800,
    height=300
)

text = alt.Chart(npidf).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-135,
    size=13
).encode(
    alt.X('date:T',axis=alt.Axis(title=None)),
    text='event',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
)

In [ ]:
#making background shading to represent different variants
cutoff = pd.DataFrame({
     "variant" : ["ancestral", "Alpha", "Delta", "Omicron"],
    'start': ["2020-02-01","2021-03-29", "2021-06-25", "2021-12-02"],
    'stop': ["2021-04-05","2021-06-28", "2021-12-31", "2022-03-8"],
})

areas_bot = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.1
).encode(
    x='start:T',
    x2='stop:T',
    y=alt.value(0),  # pixels from top
    y2=alt.value(-20),  # pixels from top
   
).transform_filter((datum.variant != "Alpha") & (datum.variant != "Omicron"))


areas_top = alt.Chart(
    cutoff.reset_index()
).mark_rect(
    opacity=0.1
).encode(
    x='start:T',
    x2='stop:T',
    y=alt.value(-10),  # pixels from top
    y2=alt.value(-30),  # pixels from top
   
).transform_filter((datum.variant != "ancestral") & (datum.variant != "Delta"))

text_variant_bot = alt.Chart(cutoff.reset_index()).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-155,
    size=13
).encode(
    alt.X('start:T',axis=alt.Axis(title=None)),
    text='variant',
    color=alt.value('#000000')
).properties(
    width=800,
    height=300
).transform_filter((datum.variant != "Alpha") & (datum.variant != "Omicron"))

text_variant_top = alt.Chart(cutoff.reset_index()).mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-175,
    size=13
).encode(
    alt.X('start:T',axis=alt.Axis(title=None)),
    text='variant',
    color=alt.value('#000000')
).transform_filter((datum.variant != "ancestral") & (datum.variant != "Delta"))



variant_freq = text_variant_bot + text_variant_top + areas_bot + areas_top




In [ ]:
jumps_df = pd.read_csv("../analysis/data-files/migration_jumps_df.csv")
root_df = pd.read_csv("../analysis/data-files/root_states_df.csv")
persistance_df = pd.read_csv("../analysis/data-files/persistance_df.csv")


In [ ]:
jumps_df.rename(columns={'migration_direction': 'Migration Direction'}, inplace=True)
jumps_df['Migration Direction'][jumps_df['Migration Direction'] == "North_King_County-to-South_King_County"] = "North to South King County"
jumps_df["Migration Direction"][jumps_df['Migration Direction'] == "South_King_County-to-North_King_County"] = "South to North King County"

root_df.rename(columns={'region': 'Region'}, inplace=True)


In [ ]:
persistance_df.rename(columns={'child_host': 'Region'}, inplace=True)
persistance_df.Region[persistance_df.Region == "North_King_County"] = "North King County"
persistance_df.Region[persistance_df.Region == "South_King_County"] = "South King County"

In [ ]:
error_bars_pr = alt.Chart(persistance_df).mark_errorbar(extent='ci').encode(
  x=alt.X('year-month:T', axis=alt.Axis( grid=False,tickCount = "month",  format="%b %Y")),
  y=alt.Y('persistence_time:Q',axis=alt.Axis(title="Length of Local Transmission (in days)", grid=False)), 
  color = alt.Color("Region:N")
).properties(
    width=1000,
    height=300
)

points_pr = alt.Chart(persistance_df).mark_point(filled=True,  opacity = 1, size = 100).encode(
  x=alt.X('year-month:T'),
  y=alt.Y('persistence_time:Q', aggregate='mean'),
    color = alt.Color("Region:N",  legend=alt.Legend(offset = -220, labelFontSize = 12, titleFontSize = 12))
).properties(
    width=1000,
    height=300
)

lineplot_pr =  alt.Chart(persistance_df).mark_line(interpolate='monotone', opacity = 0.35).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mean(persistence_time)'),
    color=alt.Color('Region:N')).properties(
    width=1000,
    height=300
)


persistence = error_bars_pr + points_pr +lineplot_pr + rule + text + variant_freq
persistence

In [ ]:
error_bars = alt.Chart(persistance_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('persistence_time:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Length of Local Transmission (in days)", grid = False,orient = "right")),
  x=alt.X('Region:N', axis=alt.Axis(labels=False)), color = alt.Color("Region:N", legend = None)
)

points = alt.Chart(persistance_df).mark_point(filled=True, color='black', size = 60).encode(
  y=alt.Y('persistence_time:Q', aggregate='mean'),
  x=alt.X('Region:N', axis=alt.Axis(title=None)),  color = alt.Color("Region:N", legend = None)
)

ave_persist = error_bars + points
#ave.save("jumps_error.png")
ave_persist

In [ ]:
persist = persistence | ave_persist
persist

In [ ]:
lineplot3 =  alt.Chart(jumps_df, width = 750).mark_line(interpolate='monotone', clip = True).encode(
    x=alt.X('year-month:T',axis=alt.Axis(tickCount = "month", grid=False,  format="%b %Y")),
    y=alt.Y('mean(mig_per_bl)',axis=alt.Axis(title="Number of Migration Events (Normalized)", grid=False)),
    color=alt.Color('Migration Direction:N', legend=alt.Legend(offset = -160, labelFontSize = 12, titleFontSize = 12), scale=alt.Scale(scheme='accent'))).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.mig_per_bl < 150)
)


band3 = alt.Chart(jumps_df).mark_errorband(extent='ci', interpolate='monotone', clip = True).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mig_per_bl',axis=alt.Axis(title="", grid=False)), 
    color =alt.Color('Migration Direction:N', scale=alt.Scale(scheme='accent'))).properties(
    width=1000,
    height=300
).transform_filter(
    (datum.mig_per_bl < 150)
)

jumps_per_bl = lineplot3 + band3 + rule + text + variant_freq
jumps_per_bl

In [ ]:
error_bars = alt.Chart(jumps_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('mig_per_bl:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Number of Migration Events",grid = False, orient = "right")),
  x=alt.X('Migration Direction:N', axis=alt.Axis(labels=False)), color = alt.Color("Migration Direction:N", legend = None, scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
)

points = alt.Chart(jumps_df).mark_point(filled=True, color='black', size = 100).encode(
  y=alt.Y('mig_per_bl:Q', aggregate='mean'),
  x=alt.X('Migration Direction:N', axis=alt.Axis(title=None)),  color = alt.Color("Migration Direction:N", legend = None, scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
)

ave_jumps = error_bars + points
#ave.save("jumps_error.png")
ave_jumps

In [ ]:
error_bars = alt.Chart(jumps_df).mark_errorbar(extent='ci', clip = True).encode(
  x=alt.X('year-month:T', axis=alt.Axis( grid=False, tickCount = "month", format="%b %Y")),
  y=alt.Y('mig_per_bl:Q',axis=alt.Axis(title="Number of Migration Events (Normalized)", grid=False), scale = alt.Scale(domain = (0,100))), color = alt.Color("Migration Direction:N", scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
).properties(
    width=1000,
    height=300
)

points = alt.Chart(jumps_df).mark_point(filled=True,  opacity = 1, size = 100, clip = True).encode(
  x=alt.X('year-month:T'),
  y=alt.Y('mig_per_bl:Q', aggregate='mean', scale = alt.Scale(domain = (0,100))),
    color = alt.Color("Migration Direction:N",  legend=alt.Legend(offset = -275, labelFontSize = 12, titleFontSize = 12),scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))
).properties(
    width=1000,
    height=300
)

lineplot4 =  alt.Chart(jumps_df).mark_line(interpolate='monotone', opacity = 0.35, clip = True).encode(
    x=alt.X('year-month:T'),
    y=alt.Y('mean(mig_per_bl)', scale = alt.Scale(domain = (0,100))),
    color=alt.Color('Migration Direction:N', scale=alt.Scale(domain = ['North to South King County', 'South to North King County'], range = ['#9461bd',"#2ca02c"]))).properties(
    width=1000,
    height=300
)


jumps_per_bl = error_bars + points +lineplot4 + rule + text + variant_freq
jumps_per_bl

In [ ]:
jumps = jumps_per_bl |ave_jumps
jumps

In [ ]:
root_states = alt.Chart(root_df).mark_bar(size=30).encode(
    alt.X('year-month:T', axis=alt.Axis(title="", grid=False, format="%B %Y")), 
    alt.Color('Region:N', legend = None),
    alt.Y("mean(mig_per_bl)", stack="normalize", title='Cluster Root States',axis=alt.Axis(offset=10 ,format='%'))).properties(
    width=1000,
    height=300
)
root_states= root_states + variant_freq
root_states.configure_axis(
    labelFontSize=14,
    titleFontSize=14
)

In [ ]:
error_bars = alt.Chart(root_df).mark_errorbar(extent='ci').encode(
  y=alt.Y('mig_per_bl:Q', scale=alt.Scale(zero=False), axis=alt.Axis(title="Average Cluster Root States", grid = False, orient = "right")),
  x=alt.X('Region:N', axis=alt.Axis(labels=False)), color = alt.Color("Region:N", legend = None)
)

points = alt.Chart(root_df).mark_point(filled=True, color='black', size = 100).encode(
  y=alt.Y('mig_per_bl:Q', aggregate='mean'),
  x=alt.X('Region:N', axis=alt.Axis(title=None)),  color = alt.Color("Region:N", legend = None)
)

ave_root = error_bars + points
#ave.save("jumps_error.png")
ave_root

In [ ]:
root = root_states | ave_root
root

In [ ]:
# Add labels to each chart
persist = persist.properties(title='A')
root = root.properties(title='B')
jumps = jumps.properties(title='C')


figure4 = (persist & root & jumps).resolve_scale(color = "independent", x = "shared").configure_axis(
    labelFontSize=12,
    titleFontSize=14
)
figure4.configure_title(
    anchor='start', fontSize= 30
).configure_axis(
    labelFontSize=14,
    titleFontSize=14
).configure_legend(
    labelFontSize = 14)